IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M2\NO_FAULT2


VARIABILE RCOU POTENZA MOTORI

In [3]:
pd.set_option('display.max_rows',700)

In [4]:
#andiamo a prendere una potenza media di 1400 per considerare il momento il cui inizia/finisce il volo
rcou_m2 = pd.read_csv("RCOU.csv")
rcou_m2 = rcou_m2.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

rcou = rcou_m2[((rcou_m2['C9'] >= 1400) & (rcou_m2['C10'] >= 1400) & (rcou_m2['C11'] >= 1400) & (rcou_m2['C12'] >= 1400) & (rcou_m2['C13'] >= 1400) & (rcou_m2['C14']>= 1400))]

display(rcou_m2)

,TimeUS,C9,C10,C11,C12,C13,C14
0,21377588,1000,1000,1000,1000,1000,1000
1,21443127,1000,1000,1000,1000,1000,1000
2,21559486,1000,1000,1000,1000,1000,1000
3,21781420,1000,1000,1000,1000,1000,1000
4,21791809,1000,1000,1000,1000,1000,1000
...,...,...,...,...,...,...,...
14807,266185051,0,0,0,0,0,0
14808,266195241,0,0,0,0,0,0
14809,266205305,0,0,0,0,0,0
14810,266215473,0,0,0,0,0,0


In [5]:
#xkf utilizzato per fare poi il confronto con i valori di roll, pitch e yaw in att
xkf1_m2 = pd.read_csv("XKF1_0.csv")
xkf1_m2 = xkf1_m2.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m2 = xkf1_m2[((xkf1_m2['TimeUS'] >= 21377588) & (xkf1_m2['TimeUS'] <= 266225648))]
print(xkf1_m2)

          TimeUS   Roll  Pitch     Yaw
1       21440633  -0.51  -1.37  355.21
2       21557359  -0.51  -1.37  355.22
3       21779295  -0.51  -1.37  355.23
4       21789580  -0.51  -1.37  355.24
5       21800259  -0.51  -1.37  355.24
...          ...    ...    ...     ...
14808  266182424  74.09  -6.28  262.59
14809  266193328  74.18  -6.62  262.50
14810  266203343  74.27  -6.94  262.39
14811  266213379  74.37  -7.24  262.25
14812  266223445  74.46  -7.57  262.07

[14812 rows x 4 columns]


In [6]:
att_m2 = pd.read_csv("ATT.csv")
att_m2 = att_m2.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m2 = att_m2[((att_m2['TimeUS'] >= 21377588) & (att_m2['TimeUS'] <= 266225648))]
print(att_m2)

          TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw     Yaw
1       21442252     0.00  -0.51       0.0  -1.37  355.20  355.22
2       21558785     0.00  -0.51       0.0  -1.37  355.21  355.23
3       21780618     0.00  -0.51       0.0  -1.37  355.23  355.24
4       21791227     0.00  -0.51       0.0  -1.37  355.23  355.25
5       21800933     0.00  -0.51       0.0  -1.37  355.23  355.25
...          ...      ...    ...       ...    ...     ...     ...
14810  266184106    -0.01  74.09       0.0  -6.28  263.26  262.60
14811  266194335    -0.01  74.18       0.0  -6.62  263.17  262.51
14812  266204442    -0.01  74.27       0.0  -6.94  263.06  262.40
14813  266214607    -0.01  74.37       0.0  -7.24  262.92  262.26
14814  266224840    -0.01  74.46       0.0  -7.57  262.74  262.08

[14814 rows x 7 columns]


In [7]:
#controllo corrispondenza valori roll, pitch, yaw
att_m2 = pd.merge_asof(att_m2, xkf1_m2, on="TimeUS", by=["Roll","Pitch","Yaw"], tolerance=100000)
print(att_m2)

          TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw     Yaw
0       21442252     0.00  -0.51       0.0  -1.37  355.20  355.22
1       21558785     0.00  -0.51       0.0  -1.37  355.21  355.23
2       21780618     0.00  -0.51       0.0  -1.37  355.23  355.24
3       21791227     0.00  -0.51       0.0  -1.37  355.23  355.25
4       21800933     0.00  -0.51       0.0  -1.37  355.23  355.25
...          ...      ...    ...       ...    ...     ...     ...
14809  266184106    -0.01  74.09       0.0  -6.28  263.26  262.60
14810  266194335    -0.01  74.18       0.0  -6.62  263.17  262.51
14811  266204442    -0.01  74.27       0.0  -6.94  263.06  262.40
14812  266214607    -0.01  74.37       0.0  -7.24  262.92  262.26
14813  266224840    -0.01  74.46       0.0  -7.57  262.74  262.08

[14814 rows x 7 columns]


In [8]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_1_m2 = pd.read_csv("ESC_1.csv")
esc_1_m2 = esc_1_m2.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m2 = esc_1_m2[((esc_1_m2['TimeUS'] >= 21377588) & (esc_1_m2['TimeUS'] <= 266225648))]
print(esc_1_m2)

        TimeUS  RPM  Curr
0     21839706    0  0.04
1     21942259    0  0.04
2     22054419    0  0.04
3     22164593    0  0.04
4     22265674    0  0.03
..         ...  ...   ...
928  254509624    0  0.02
929  254617493    0  0.03
930  254724316    0  0.02
931  254832844    0  0.02
932  254940415    0  0.02

[933 rows x 3 columns]


In [9]:
#valori di accelerazione e del giroscopio
imu_2_m2 = pd.read_csv("IMU_2.csv")
imu_2_m2 = imu_2_m2.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m2 = imu_2_m2[((imu_2_m2['TimeUS'] >= 21377588) & (imu_2_m2['TimeUS'] <= 266225648))]
print(imu_2_m2)

          TimeUS      GyrX      GyrY      GyrZ      AccX       AccY      AccZ
1       21379280 -0.000143 -0.003713 -0.000835 -0.317570   0.168200 -9.851592
2       21405648 -0.001483 -0.002965  0.000405 -0.309169   0.190218 -9.865791
3       21442085  0.003247  0.004227 -0.001391 -0.323399   0.219677 -9.820618
4       21444703  0.003511  0.004961 -0.000932 -0.323141   0.220400 -9.813485
5       21463665  0.001315  0.002153  0.001588 -0.313926   0.210318 -9.826613
...          ...       ...       ...       ...       ...        ...       ...
57448  266214413  0.139755 -0.387664  0.438880  0.058760 -11.477650 -3.189833
57449  266217260  0.133286 -0.399660  0.430425  0.029484 -11.449310 -3.220269
57450  266220378  0.131108 -0.403541  0.428677 -0.001193 -11.439130 -3.249021
57451  266221876  0.122037 -0.413943  0.424900 -0.057513 -11.436480 -3.310098
57452  266224661  0.111823 -0.420368  0.423021 -0.098880 -11.438530 -3.363329

[57452 rows x 7 columns]


In [10]:
#sincronizzazione dei tempi tra att e imu_2
m2_nofault = pd.merge_asof(att_m2, imu_2_m2, on="TimeUS",  tolerance= 100000)
print(m2_nofault)

          TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw     Yaw      GyrX  \
0       21442252     0.00  -0.51       0.0  -1.37  355.20  355.22  0.003247   
1       21558785     0.00  -0.51       0.0  -1.37  355.21  355.23 -0.000854   
2       21780618     0.00  -0.51       0.0  -1.37  355.23  355.24  0.002060   
3       21791227     0.00  -0.51       0.0  -1.37  355.23  355.25  0.004058   
4       21800933     0.00  -0.51       0.0  -1.37  355.23  355.25  0.003012   
...          ...      ...    ...       ...    ...     ...     ...       ...   
14809  266184106    -0.01  74.09       0.0  -6.28  263.26  262.60  0.144341   
14810  266194335    -0.01  74.18       0.0  -6.62  263.17  262.51  0.143419   
14811  266204442    -0.01  74.27       0.0  -6.94  263.06  262.40  0.145466   
14812  266214607    -0.01  74.37       0.0  -7.24  262.92  262.26  0.139755   
14813  266224840    -0.01  74.46       0.0  -7.57  262.74  262.08  0.111823   

           GyrY      GyrZ      AccX       AccY     

In [11]:
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m2_nofault = pd.merge_asof(esc_1_m2, m2_nofault, on="TimeUS",  tolerance= 100000)
print(m2_nofault)

        TimeUS  RPM  Curr  DesRoll  Roll  DesPitch  Pitch  DesYaw     Yaw  \
0     21839706    0  0.04     0.00 -0.51       0.0  -1.37  355.23  355.24   
1     21942259    0  0.04     0.00 -0.51       0.0  -1.37  355.20  355.22   
2     22054419    0  0.04     0.00 -0.51       0.0  -1.37  355.19  355.21   
3     22164593    0  0.04     0.00 -0.50       0.0  -1.37  355.20  355.22   
4     22265674    0  0.03     0.00 -0.51       0.0  -1.38  355.21  355.23   
..         ...  ...   ...      ...   ...       ...    ...     ...     ...   
928  254509624    0  0.02    -0.01  2.37       0.0  -1.93  357.34  357.32   
929  254617493    0  0.03    -0.01  2.43       0.0  -1.96  357.40  357.38   
930  254724316    0  0.02    -0.01  2.43       0.0  -1.93  357.43  357.41   
931  254832844    0  0.02    -0.01  2.46       0.0  -1.91  357.46  357.44   
932  254940415    0  0.02    -0.01  2.46       0.0  -1.89  357.49  357.47   

         GyrX      GyrY      GyrZ      AccX      AccY      AccZ  
0    0.00

In [12]:
#aggiornamento del tempo ponendo a 0 il momento in cui il volo inizia per noi e settaggio della variabile guasto
m2_nofault["TimeUS"] = m2_nofault["TimeUS"] - m2_nofault.iloc[0]["TimeUS"]
m2_nofault["TimeUS"] = m2_nofault["TimeUS"].astype(int)
m2_nofault["Guasto"] = 0
print(m2_nofault)

        TimeUS  RPM  Curr  DesRoll  Roll  DesPitch  Pitch  DesYaw     Yaw  \
0            0    0  0.04     0.00 -0.51       0.0  -1.37  355.23  355.24   
1       102553    0  0.04     0.00 -0.51       0.0  -1.37  355.20  355.22   
2       214713    0  0.04     0.00 -0.51       0.0  -1.37  355.19  355.21   
3       324887    0  0.04     0.00 -0.50       0.0  -1.37  355.20  355.22   
4       425968    0  0.03     0.00 -0.51       0.0  -1.38  355.21  355.23   
..         ...  ...   ...      ...   ...       ...    ...     ...     ...   
928  232669918    0  0.02    -0.01  2.37       0.0  -1.93  357.34  357.32   
929  232777787    0  0.03    -0.01  2.43       0.0  -1.96  357.40  357.38   
930  232884610    0  0.02    -0.01  2.43       0.0  -1.93  357.43  357.41   
931  232993138    0  0.02    -0.01  2.46       0.0  -1.91  357.46  357.44   
932  233100709    0  0.02    -0.01  2.46       0.0  -1.89  357.49  357.47   

         GyrX      GyrY      GyrZ      AccX      AccY      AccZ  Guasto  
0

In [13]:
path_file = path_file.replace(r"M2\NO_FAULT2", "")
os.chdir(path_file)
m2_nofault.to_csv('m2_nofault.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
